# Estimating evolution circuit runtimes

The aim of this notebook is to explain the noisy Hamiltonian simulation results. In these results, the trace distance plateaus at a specific value; and for the Gray code encoding, there is actually an inflection point, i.e. trace distance decreases up to a point, but then gets worse. 

A reasonable explanation for these features would be:
 - the plateaus occur when the output state is fully decohered (we can check this by computing the trace distance of the true output state with the maximally mixed state, and see if we get a similar value)
 - the dip in the Gray code version occurs when the circuit execution times start to surpass coherence times
 
Let's do some very rough estimates and see if we're in the ballpark.

In [1]:
import numpy as np
np.warnings.filterwarnings('ignore')

import pickle

from scipy.linalg import expm

# Everything we need from Qiskit
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute, Aer
from qiskit.quantum_info import Pauli

from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.components.initial_states import Custom

import qutip as qt

import sys
sys.path.append("../src/")
from hamiltonian import *
from utils import *
from qiskit_circuits import *

## Computing trace distances

Here, we compare the trace distance of the true expected state after Hamiltonian evolution to the maximally mixed state, in order to explain the plateau.

In [2]:
# Perform unitary evolution 
def unitary_evolution(ham, t):
    return expm(-1j * ham * t)

In [3]:
H = DenseEncodingHamiltonian(N_states=4)
gc_ham_rep = reduce(lambda x, y: x + y, [p[1] * get_pauli_matrix(p[0]) for p in H.pauli_coeffs.items()])

uniform_gc = 0.5 * np.array([[1], [1], [1], [1]])
maximally_mixed_gc = (1./4)*np.eye(4)

output_state_gc = unitary_evolution(gc_ham_rep, 1) @ uniform_gc
td = qt.tracedist(qt.Qobj(output_state_gc), qt.Qobj(maximally_mixed_gc))

print(f"Expected trace distance for Gray code after decoherence: {td}")

Expected trace distance for Gray code after decoherence: 0.7500000000000009


In [4]:
jw = SparseEncodingHamiltonian(N_states=4)
jw_ham_rep = reduce(lambda x, y: x + y, [p[1] * get_pauli_matrix(p[0]) for p in jw.pauli_coeffs.items()])

uniform_jw = 0.5 * np.array([[0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0]]).reshape((16, 1))
maximally_mixed_jw = (1./16)*np.eye(16)

output_state_jw = unitary_evolution(jw_ham_rep, 1) @ uniform_jw
td = qt.tracedist(qt.Qobj(output_state_jw), qt.Qobj(maximally_mixed_jw))

print(f"Expected trace distance: {td}")

Expected trace distance: 0.9375000000000007


From the graphics in the other file, these are reasonably close to the values of the plateaued trace distance. The Gray code one is maybe slightly lower than we would expect, but still around the right place.

## Circuit run time estimates

In [5]:
# Numbers pulled from Vigo device model from the noisy simulations
qubit0_t2 = 17.14151362421807e-6
qubit1_t2 = 111.42172460237252e-6
qubit2_t2 = 141.43954844693835e-6
qubit3_t2 = 120.90398756232938e-6

# Gate time estimates pulled from Vigo a different day; just an estimate, we'll average
mean_1qubit_gate_length = 71.1e-9
cnot_01 = 348.4e-9
cnot_10 = 384e-9
cnot_12 = 227.5e-9
cnot_21 = 263.11e-9
mean_cnot = (cnot_01 + cnot_10 + cnot_12 + cnot_21) / 4

# Arbitrary simulation time
T = 1
backend = Aer.get_backend('statevector_simulator')

Determine the evolution circuit for the Gray code version - then count the time steps and multiple by the number of Trotter steps to get an idea of the run time.

In [6]:
H_gc = DenseEncodingHamiltonian(N_states=4)

weighted_paulis = [(v, Pauli.from_label(k)) for (k, v) in list(H_gc.pauli_coeffs.items())]
my_pauli = WeightedPauliOperator(weighted_paulis)

# Boilerplate code from tutorial
q = QuantumRegister(H_gc.N_qubits)
c = ClassicalRegister(H_gc.N_qubits)

circuit = QuantumCircuit(q, c)

circuit.h(q)

circuit += my_pauli.evolve(
    None, evo_time=T, num_time_slices=1,
    quantum_registers=q,
    expansion_mode='trotter'
)

circuit.measure(q, c)

In [7]:
circuit.decompose().draw()

┌──────────┐                         ┌─────────────┐                 »
q0_0: |0>┤ U2(0,pi) ├──■───────────────────■──┤ U1(-2.8433) ├──────────────■──»
         ├──────────┤┌─┴─┐┌─────────────┐┌─┴─┐├─────────────┤┌──────────┐┌─┴─┐»
q0_1: |0>┤ U2(0,pi) ├┤ X ├┤ U1(-9.8433) ├┤ X ├┤ U1(-16.843) ├┤ U2(0,pi) ├┤ X ├»
         └──────────┘└───┘└─────────────┘└───┘└─────────────┘└──────────┘└───┘»
 c0_0: 0 ═════════════════════════════════════════════════════════════════════»
                                                                              »
 c0_1: 0 ═════════════════════════════════════════════════════════════════════»
                                                                              »
«                         ┌──────────┐                                       »
«q0_0: ────────────────■──┤ U2(0,pi) ├───────────────────────────────────────»
«      ┌────────────┐┌─┴─┐├──────────┤┌──────────┐┌─────────────┐┌──────────┐»
«q0_1: ┤ U1(7.8262) ├┤ X ├┤ U2(0,pi) ├┤ U2(0,pi) ├┤ U1(-7.8262) ├┤ U2(0,pi) ├»
«      └────────────┘└───┘└──────────┘└──────────┘└─────────────┘└──────────┘»
«c0_0: ══════════════════════════════════════════════════════════════════════»
«                                                                            »
«c0_1: ══════════════════════════════════════════════════════════════════════»
«                                                                            »
«                              ┌──────────┐┌──────────┐┌─────────────┐»
«q0_0: ──■──────────────────■──┤ U2(0,pi) ├┤ U2(0,pi) ├┤ U1(-15.628) ├»
«      ┌─┴─┐┌────────────┐┌─┴─┐└───┬─┬────┘└──────────┘└─────────────┘»
«q0_1: ┤ X ├┤ U1(7.0547) ├┤ X ├────┤M├────────────────────────────────»
«      └───┘└────────────┘└───┘    └╥┘                                »
«c0_0: ═════════════════════════════╬═════════════════════════════════»
«                                   ║                                 »
«c0_1: ═════════════════════════════╩═════════════════════════════════»
«                                                                     »
«      ┌──────────┐┌─┐
«q0_0: ┤ U2(0,pi) ├┤M├
«      └──────────┘└╥┘
«q0_1: ─────────────╫─
«                   ║ 
«c0_0: ═════════════╩═
«                     
«c0_1: ═══════════════
«

In [8]:
# Count up layers of depth; ignore the first row of U2 because that's just
# the Hadamards for state prep and is not repeated in multiple Trotter steps
# Calculated by inspection of the above circuit 
gc_circuit_time_estimate = 13*mean_1qubit_gate_length + 6*cnot_01

In [9]:
print(f"Coherence time qubit 0 (μs): {1e6*qubit0_t2:.2f}")
print(f"Coherence time qubit 1 (μs): {1e6*qubit1_t2:.2f}")
print()
for trotter_steps in range(1, 45):
    circuit_time_estimate = mean_1qubit_gate_length + trotter_steps*gc_circuit_time_estimate
    print(f"{trotter_steps} Trotter steps, execution time estimate (μs) {1e6*circuit_time_estimate:.2f}")

Coherence time qubit 0 (μs): 17.14
Coherence time qubit 1 (μs): 111.42

1 Trotter steps, execution time estimate (μs) 3.09
2 Trotter steps, execution time estimate (μs) 6.10
3 Trotter steps, execution time estimate (μs) 9.12
4 Trotter steps, execution time estimate (μs) 12.13
5 Trotter steps, execution time estimate (μs) 15.14
6 Trotter steps, execution time estimate (μs) 18.16
7 Trotter steps, execution time estimate (μs) 21.17
8 Trotter steps, execution time estimate (μs) 24.19
9 Trotter steps, execution time estimate (μs) 27.20
10 Trotter steps, execution time estimate (μs) 30.22
11 Trotter steps, execution time estimate (μs) 33.23
12 Trotter steps, execution time estimate (μs) 36.25
13 Trotter steps, execution time estimate (μs) 39.26
14 Trotter steps, execution time estimate (μs) 42.28
15 Trotter steps, execution time estimate (μs) 45.29
16 Trotter steps, execution time estimate (μs) 48.31
17 Trotter steps, execution time estimate (μs) 51.32
18 Trotter steps, execution time estima

So the time estimates are still less than the coherence time of the 2nd qubit until around 36 Trotter steps. This is about to point where things are beginning to plateau, so this checks out.

Now for the Jordan-Wigner version...

In [10]:
H_jw = SparseEncodingHamiltonian(N_states=4, qiskit_order=True)
weighted_paulis = [(v, Pauli.from_label(k)) for (k, v) in list(H_jw.pauli_coeffs.items())]
my_pauli = WeightedPauliOperator(weighted_paulis)

# To prepare the uniform superposition, run the sparse variational ansatz with the following parameters
θ_1 = 2 * np.pi / 3
θ_2 = 2 * np.arccos(1/np.sqrt(3)) 
θ_3 = 2 * np.arccos(1/(np.sqrt(3) * np.sin(θ_2 / 2)))
params = [θ_1, θ_2, θ_3]

# Construct uniform superposition over spherical coordinates
q = QuantumRegister(H_jw.N_qubits)
c = ClassicalRegister(H_jw.N_qubits)

circuit = QuantumCircuit(q, c)
circuit.x(q[0])
circuit.ry(θ_1, q[1])
circuit.cx(q[1], q[0])
circuit.cry(θ_2, q[1], q[2])
circuit.cx(q[2], q[1])
circuit.cry(θ_3, q[2], q[3])
circuit.cx(q[3], q[2])

circuit += my_pauli.evolve(
    None, evo_time=T, num_time_slices=1,
    quantum_registers=q,
    expansion_mode='trotter'
)

circuit.decompose().draw()

┌─────────────┐  ┌───┐┌─────────────┐┌─────────────────────┐     »
q2_0: |0>──┤ U3(pi,0,pi) ├──┤ X ├┤ U1(0.43658) ├┤ U3(pi/2,-pi/2,pi/2) ├─────»
          ┌┴─────────────┴┐ └─┬─┘└─────────────┘└─────────────────────┘     »
q2_1: |0>─┤ R(2pi/3,pi/2) ├───■─────────■────────────────────────────────■──»
         ┌┴───────────────┴┐          ┌─┴─┐       ┌──────────────────┐ ┌─┴─┐»
q2_2: |0>┤ U3(0.95532,0,0) ├──────────┤ X ├───────┤ U3(-0.95532,0,0) ├─┤ X ├»
         └─┬──────────────┬┘          └───┘       └──────────────────┘ └───┘»
q2_3: |0>──┤ U3(pi/4,0,0) ├─────────────────────────────────────────────────»
           └──────────────┘                                                 »
 c1_0: 0 ═══════════════════════════════════════════════════════════════════»
                                                                            »
 c1_1: 0 ═══════════════════════════════════════════════════════════════════»
                                                                            »
 c1_2: 0 ═══════════════════════════════════════════════════════════════════»
                                                                            »
 c1_3: 0 ═══════════════════════════════════════════════════════════════════»
                                                                            »
«                                                                        »
«q2_0: ──────────────────────────────────────────────────────────────────»
«      ┌───┐┌────────────┐┌─────────────────────┐                        »
«q2_1: ┤ X ├┤ U1(-12.25) ├┤ U3(pi/2,-pi/2,pi/2) ├────────────────────────»
«      └─┬─┘└────────────┘└─────────────────────┘     ┌───┐┌────────────┐»
«q2_2: ──■────────■────────────────────────────────■──┤ X ├┤ U1(-19.25) ├»
«               ┌─┴─┐        ┌───────────────┐   ┌─┴─┐└─┬─┘├────────────┤»
«q2_3: ─────────┤ X ├────────┤ U3(-pi/4,0,0) ├───┤ X ├──■──┤ U1(-26.25) ├»
«               └───┘        └───────────────┘   └───┘     └────────────┘»
«c1_0: ══════════════════════════════════════════════════════════════════»
«                                                                        »
«c1_1: ══════════════════════════════════════════════════════════════════»
«                                                                        »
«c1_2: ══════════════════════════════════════════════════════════════════»
«                                                                        »
«c1_3: ══════════════════════════════════════════════════════════════════»
«                                                                        »
«                                                                              »
«q2_0: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q2_1: ────────────────────────────────────────────────────────────────────────»
«      ┌─────────────────────┐                         ┌──────────────────────┐»
«q2_2: ┤ U3(pi/2,-pi/2,pi/2) ├──■───────────────────■──┤ U3(-pi/2,-pi/2,pi/2) ├»
«      ├─────────────────────┤┌─┴─┐┌─────────────┐┌─┴─┐├──────────────────────┤»
«q2_3: ┤ U3(pi/2,-pi/2,pi/2) ├┤ X ├┤ U1(-11.341) ├┤ X ├┤ U3(-pi/2,-pi/2,pi/2) ├»
«      └─────────────────────┘└───┘└─────────────┘└───┘└──────────────────────┘»
«c1_0: ════════════════════════════════════════════════════════════════════════»
«                                                                              »
«c1_1: ════════════════════════════════════════════════════════════════════════»
«                                                                              »
«c1_2: ════════════════════════════════════════════════════════════════════════»
«                                                                              »
«c1_3: ════════════════════════════════════════════════════════════════════════»
«                                                                              »
«                                

In [11]:
jw_circuit_time_estimate = 17*mean_cnot + 18*mean_1qubit_gate_length

In [12]:
print(f"Coherence time qubit 0 (μs): {1e6*qubit0_t2:.2f}")
print(f"Coherence time qubit 1 (μs): {1e6*qubit1_t2:.2f}")
print(f"Coherence time qubit 2 (μs): {1e6*qubit2_t2:.2f}")
print(f"Coherence time qubit 3 (μs): {1e6*qubit3_t2:.2f}")
print()
for trotter_steps in range(1, 20):
    circuit_time_estimate = mean_1qubit_gate_length + trotter_steps*jw_circuit_time_estimate
    print(f"{trotter_steps} Trotter steps, execution time estimate (μs) {1e6*circuit_time_estimate:.2f}")

Coherence time qubit 0 (μs): 17.14
Coherence time qubit 1 (μs): 111.42
Coherence time qubit 2 (μs): 141.44
Coherence time qubit 3 (μs): 120.90

1 Trotter steps, execution time estimate (μs) 6.55
2 Trotter steps, execution time estimate (μs) 13.03
3 Trotter steps, execution time estimate (μs) 19.50
4 Trotter steps, execution time estimate (μs) 25.98
5 Trotter steps, execution time estimate (μs) 32.46
6 Trotter steps, execution time estimate (μs) 38.94
7 Trotter steps, execution time estimate (μs) 45.41
8 Trotter steps, execution time estimate (μs) 51.89
9 Trotter steps, execution time estimate (μs) 58.37
10 Trotter steps, execution time estimate (μs) 64.85
11 Trotter steps, execution time estimate (μs) 71.32
12 Trotter steps, execution time estimate (μs) 77.80
13 Trotter steps, execution time estimate (μs) 84.28
14 Trotter steps, execution time estimate (μs) 90.76
15 Trotter steps, execution time estimate (μs) 97.23
16 Trotter steps, execution time estimate (μs) 103.71
17 Trotter steps,

The execution time for the JW circuits start to surpass the first coherence time after like 2 Trotter steps. 2 Trotter steps isn't enough to give good simulation accuracy in the first place, and things just go downhill from there.